In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
def f1(true, pred, return_intersection=False):
    true_set = set(true)
    pred_set = set(pred)
    intersection = true_set & pred_set
    correct = len(intersection)
    if correct == 0:
        if return_intersection:
            return 0, intersection
        return 0
    precision = correct / len(pred_set)
    recall = correct / len(true_set)
    f1_ = 2 * precision * recall / (precision + recall)
    if return_intersection:
        return f1_, intersection
    return f1_

In [3]:
def src(true, pred):
    pos_true = {t: i for i, t in enumerate(true)}
    N = len(pred)  # N = 10
    d2 = 0
    for pos_p, p in enumerate(pred):
        pos_t = pos_true[p]
        di = min(abs(pos_p - pos_t), N)
        d2 += di ** 2
    
    src_ = 1 - 6 * d2 / (N * (N ** 2 - 1))
    # src_ = 1 - d2 / 165
    return src_

In [4]:
def leaderboard_score(sub, ans):
    ans = ans.sort_values(by='涨跌幅', ascending=False)['股票代码']
    scores = [
        f1(ans[:10], sub['涨幅最大股票代码']),
        f1(ans[-10:], sub['涨幅最小股票代码']),
        src(ans, sub['涨幅最大股票代码']),
        src(ans[::-1], sub['涨幅最小股票代码'][::-1]),
    ]
    final_score = np.average(scores, weights=[0.2, 0.2, 0.3, 0.3])
    scores.append(final_score)
    scores = pd.DataFrame(
        scores,
        index=['F1_up', 'F1_down', 'RC_up', 'RC_down', 'Final'],
        columns=['scores']
    )
    return scores

In [ ]:
sub = './output/result.csv'  # 预测结果
ans = './output/bfq_stocks_428.csv'  # 真实数据
sub = pd.read_csv(sub)
ans = pd.read_csv(ans)
leaderboard_score(sub, ans)

,scores
F1_up,0.100000
F1_down,0.000000
RC_up,-4.163636
RC_down,-5.060606
Final,-2.747273
